In [1]:
## adapted from https://machinelearningmastery.com/naive-bayes-clasifier-scratch-python/
### updated to python 3.7 from v.2
## Naive Bayes classification -- currently assumes Gaussian distribution (continuous data)
## note: nothing here needs to be modified to include more or fewer test variables, unless moving between continuous/discrete variables
import random 
import csv
import math

# will only read CSV without headers or quotes
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [2]:
filename = 'bayestest1.csv'
dataset = loadCsv(filename)
#print('Loaded data file {0} with {1} rows'.format(filename, len(dataset)))
#print(dataset)

In [3]:
def splitDataset(dataset, splitRatio):
    """
    Randomly splits dataset into training set
    and testing set.
    Most common split ~60% training, ~30% testing. 
    """
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy)) #randomly selects elements from dataset
        trainSet.append(copy.pop(index)) #removes index element from copy, adds it to training set
    return [trainSet, copy] #remaining elements in copy act as the test set

In [4]:
### Summarize data
## Generally: summarizes properties in the training set, and calculates probabilities. 


def separateByClass(dataset):
    '''
    Separates the training dataset by class (e.g. open/closed),
    so that stats can be determined for each class. 
    Maps each class value to a list of instances belonging to that class,
    and then sorts the entire dataset of instances into appropriate list.
    
    Note: this code assumes that the last attribute (last .csv column) is the class value. 
    
    Returns: Dict/map of class values to lists of data elements. 
    '''
    separated = {} #empty dictionary/value mapping
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [5]:
def mean(numbers):
    """
    Calculates mean of input. 
    Used alongside stdev for constructing Gaussian for predictions. 
    """
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [6]:
def summarize(dataset):
    """
    Summary of mean/stdev for whole dataset.
    Used within loop for summary by class function. 
    """
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries 

def summarizeByClass(dataset):
    """
    Separates dataset by class (open/close) 
    and performs summary stats within class. 
    """
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [7]:
### Predictions
## calculating the probability that a given data instance belongs to each class, 
## and selecting the class with the largest probability as the prediction. 

def calculateProbability(x, mean, stdev):
    """
    gaussian probability function for a given set of data/mean/stdev. 
    Conditional probability for a given attribute value given a class (probability of an attribute
    belonging to a class).
    """
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    """
    Maps probabilities to classes/probability of a data instance belonging to a class. 
    """
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [8]:
def predict(summaries, inputVector):
    """
    Compares probabilities of a data instance belonging to a class. 
    Returns: Most likely class, given data.
    """
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb =  None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability 
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    """
    Takes data split into test set and predicts class for each test instance. 
    Returns: Predicted classes, respective to data point.
    """
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions 

In [9]:
def getAccuracy(testSet, predictions):
    """
    Compares predicted class to actual class in the test dataset. 
    Calculates classification accuracy as percentage. 
    Returns: accuracy ration, as a percentage. 
    """
    correct = 0 
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [10]:
def main():
    """
    Putting it all together. 
    """
    filename = 'bayestest1.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    # prepare model
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))
 
main()

Split 30831 rows into train=20656 and test=10175 rows
Accuracy: 62.11302211302211%
